# Lesson 11: File I/O

<hr>

In [1]:
import os
import glob

<hr/>

Reading data in from files and then writing your results out again is one of the most common practices in scientific computing. In this tutorial, we will learn about some of Python's File I/O capabilities. We will use a PDB file as an example. The PDB file contains the crystal structure for the tetramerization domain of [p53](https://en.wikipedia.org/wiki/P53).It is stored in the  file `~/git/bootcamp/data/1OLG.pdb`. (Make sure you launch your notebook from the `~/git/bootcamp/` directory.) Note that `1OLG` is its unique [Protein Databank](http://www.rcsb.org/pdb/explore/explore.do?structureId=1olg) identifier.

## File objects

To open a file, we use the built-in `open()` function. When opening files, we should do this using **context management**. I will demonstrate how to open a file and then describe the syntax.

In [2]:

import os
import glob

with open('/Users/jschaefer/git/bootcamp/data/1OLG.pdb', 'r') as f:
    print(type(f))

<class '_io.TextIOWrapper'>


Python has a wonderful keyword, `with`. This keyword enables **context management**. Upon entry into a `with` block, variables have certain meaning. In this case, the variable `f` has the meaning of an open file, an instance of the `_io.TextIOWrapper` class. Upon exit, certain operations take place. For file objects created by opening them, the file is automatically closed upon exit, **even if there is an error**. This is important. If your program raises an exception before you have a chance to close the file, it won't get closed and you could be in trouble. If you use context management, the file will still get closed. So here is an important tip:

<div style="color: dodgerblue; text-align: center; font-weight: bold;">

Use context management using <tt>with</tt> when working with files.
    
</div>

Let's focus for a moment on the variable `f` in the above code cell. It is a Python `file` object, which has methods and attributes, just like any other object. We'll explore those in a moment, but first, let's look at how we opened the file. The first argument to `open()` is a string that has the name of the file, with the full path if necessary. The second argument is a string that says what we will be doing with the file. I.e., are we reading or writing to the file? The possible strings for this second argument are

|string | meaning|
|:------|:-------|
|`'r'` | open a text file for reading|
|`'w'` | create and open a text file for writing|
|`'a'` | append an existing text file|
|`'r+'`| open a text file for reading and writing|
|append `'b'` to any of the above | same as above, except for binary files|

We will mostly be working with text files in the bootcamp, so the first three are the most useful.  A big warning, though....


<div style="color: tomato; text-align: center; font-weight: bold;">

Trying to open an existing file with <tt>'w'</tt> will wipe it out and create a new file.
    
</div>


### Reading data out of the file with file object methods

We will focus on the methods `f.read()` and `f.readlines()`. What do they do?

|method | task|
|:------|:-------|
|`f.read()` | Read the entire contents of the file into a string|
|`f.readlines()` | Read the entire file into a list with each item being a string representing a line|

First, we'll try using the first method to get a single string with the entire contents of the file.

In [ ]:
# Read file into string
with open('/Users/jschaefer/git/bootcamp/data/1OLG.pdb', 'r') as f:
    f_str = f.read() #look at the string

# Let's look at the first 1000 characters
f_str[:10000]

'HEADER    ANTI-ONCOGENE                           13-JUN-94   1OLG              \nTITLE     HIGH-RESOLUTION SOLUTION STRUCTURE OF THE OLIGOMERIZATION             \nTITLE    2 DOMAIN OF P53 BY MULTI-DIMENSIONAL NMR                               \nCOMPND    MOL_ID: 1;                                                            \nCOMPND   2 MOLECULE: TUMOR SUPPRESSOR P53 (OLIGOMERIZATION DOMAIN);             \nCOMPND   3 CHAIN: A, B, C, D;                                                   \nCOMPND   4 ENGINEERED: YES                                                      \nSOURCE    MOL_ID: 1;                                                            \nSOURCE   2 ORGANISM_SCIENTIFIC: HOMO SAPIENS;                                   \nSOURCE   3 ORGANISM_COMMON: HUMAN;                                              \nSOURCE   4 ORGANISM_TAXID: 9606                                                 \nKEYWDS    ANTI-ONCOGENE                                                         \nEXPDTA    SOLUT

We see lots of `\n`, which signifies a new line. The backslash is known as an **escape character**, meaning that the `n` after it does not signify the letter n, but that `\n` together means a new line.

Now, let's try reading it in as a list.

In [ ]:
# Read contents of the file in as a list
with open('/Users/jschaefer/git/bootcamp/data/1OLG.pdb', 'r') as f:
    f_list = f.readlines() #look at the list of lines

# Look at the list (first ten entries)
f_list[:10]

['HEADER    ANTI-ONCOGENE                           13-JUN-94   1OLG              \n',
 'TITLE     HIGH-RESOLUTION SOLUTION STRUCTURE OF THE OLIGOMERIZATION             \n',
 'TITLE    2 DOMAIN OF P53 BY MULTI-DIMENSIONAL NMR                               \n',
 'COMPND    MOL_ID: 1;                                                            \n',
 'COMPND   2 MOLECULE: TUMOR SUPPRESSOR P53 (OLIGOMERIZATION DOMAIN);             \n',
 'COMPND   3 CHAIN: A, B, C, D;                                                   \n',
 'COMPND   4 ENGINEERED: YES                                                      \n',
 'SOURCE    MOL_ID: 1;                                                            \n',
 'SOURCE   2 ORGANISM_SCIENTIFIC: HOMO SAPIENS;                                   \n',
 'SOURCE   3 ORGANISM_COMMON: HUMAN;                                              \n']

We see that each entry is a line, including the newline character. To look at lines in files, the `rstrip()` method for strings can come it handy. It strips all whitespace, including newlines, from the end of a string.

In [5]:
f_list[0].rstrip()

'HEADER    ANTI-ONCOGENE                           13-JUN-94   1OLG'

### Reading line-by-line

What if we do not want to read the entire file into a list? For example, if a file is several gigabytes, we do not want to spend all of our RAM storing a list. Instead, we can read it line-by-line. Conveniently, the file object can be used as an iterator.

In [ ]:
# Print the first ten lines of the file
with open('/Users/jschaefer/git/bootcamp/data/1OLG.pdb', 'r') as f:
    for i, line in enumerate(f):
        print(line.rstrip())
        if i >= 5: # stop after 5 lines
            break

HEADER    ANTI-ONCOGENE                           13-JUN-94   1OLG
TITLE     HIGH-RESOLUTION SOLUTION STRUCTURE OF THE OLIGOMERIZATION
TITLE    2 DOMAIN OF P53 BY MULTI-DIMENSIONAL NMR
COMPND    MOL_ID: 1;
COMPND   2 MOLECULE: TUMOR SUPPRESSOR P53 (OLIGOMERIZATION DOMAIN);
COMPND   3 CHAIN: A, B, C, D;


Alternatively, we can use the method `f.readline()` to read a single line in the file and return it as a string.

In [ ]:
# Print the first ten lines of the file
with open('/Users/jschaefer/git/bootcamp/data/1OLG.pdb', 'r') as f:
    i = 0
    while i < 10:
        print(f.readline().rstrip()) #apply two methods to the same object 
        i += 1

HEADER    ANTI-ONCOGENE                           13-JUN-94   1OLG
TITLE     HIGH-RESOLUTION SOLUTION STRUCTURE OF THE OLIGOMERIZATION
TITLE    2 DOMAIN OF P53 BY MULTI-DIMENSIONAL NMR
COMPND    MOL_ID: 1;
COMPND   2 MOLECULE: TUMOR SUPPRESSOR P53 (OLIGOMERIZATION DOMAIN);
COMPND   3 CHAIN: A, B, C, D;
COMPND   4 ENGINEERED: YES
SOURCE    MOL_ID: 1;
SOURCE   2 ORGANISM_SCIENTIFIC: HOMO SAPIENS;
SOURCE   3 ORGANISM_COMMON: HUMAN;


Each subsequent call to `f.readline()` reads in the next line of the file. (As we read through a file, we keep moving forward in the bytes of the file and we have to use `f.seek()` to rewind.)

## Writing to a file

Writing to a file has similar syntax. We already saw how to open a file for writing. Again, context management is useful. However, before trying to open a file, we should check to make sure a file of the same name does not exist before opening it. The `os.path` module is useful. The function `os.path.isfile()` function checks to see if a file exists.

In [15]:
import os
import glob
os.path.isfile('/Users/jschaefer/git/bootcamp/data/1OLG.pdb')

True

Now that we know how to check existence of a file so we do not overwrite it, we can open and write a file.

In [16]:
if os.path.isfile('yogi.txt'):
    raise RuntimeError('File yogi.txt already exists.')

with open('yogi.txt', 'w') as f:
    f.write('When you come to a fork in the road, take it.')
    f.write('You can observe a lot by just watching.')
    f.write('I never said most of the things I said.')

RuntimeError: File yogi.txt already exists.

Note that we can use the `f.write()` method to write strings to a file. Let's look at the file contents.

In [17]:
!cat yogi.txt 

When you come to a fork in the road, take it.You can observe a lot by just watching.I never said most of the things I said.

Ah!  There are no newlines!  When writing to a file, unlike when you use the `print()` function, you must include the newline characters.  Let's try again, intentionally obliterating our first attempt.

In [11]:
with open('yogi.txt', 'w') as f:
    f.write('When you come to a fork in the road, take it.\n')
    f.write('You can observe a lot by just watching.\n')
    f.write('I never said most of the things I said.\n')
    
!cat yogi.txt

When you come to a fork in the road, take it.
You can observe a lot by just watching.
I never said most of the things I said.


That's better. Note also that `f.write()` **only** takes strings as arguments. You cannot pass numbers. They must be converted to strings first.

In [12]:
# This will result in an exception
with open('gimme_phi.txt', 'w') as f:
    f.write('The golden ratio is φ = ')
    f.write(1.61803398875)

TypeError: write() argument must be str, not float

Yup.  It must be a string.  Let's try again.

In [18]:
with open('gimme_phi.txt', 'w') as f:
    f.write('The golden ratio is φ = ')
    f.write('{phi:.8f}'.format(phi=1.61803398875))

!cat gimme_phi.txt

The golden ratio is φ = 1.61803399

That works!

## An exercise: extract atomic coordinates for first chain in tetramer

As an example on how to do file I/O, we will take the PDB file and extract only the `ATOM` records for the first chain of the tetramer and write only those entries to a new file.

It is useful to know that according to the [PDB format specification](http://www.wwpdb.org/documentation/file-format-content/format33/sect9.html#ATOM), column 21 in the `ATOM` entry gives the ID of the chain.

We also conveniently use the fact that we can have multiple files open in our `with` block, separating them with commas.

In [21]:
with open('/Users/jschaefer/git/bootcamp/data/1OLG.pdb', 'r') as f, open('atoms_chain_A.txt', 'w') as f_out:
    # Put the ATOM lines from chain A in new file
    for line in f:
        if len(line) > 21 and line[:4] == 'ATOM' and line[21] == 'B':
            f_out.write(line)

Let's see how we did!

In [22]:
!head -10 atoms_chain_A.txt

ATOM    700  N   LYS B 319     -18.744  25.982 -10.009  1.00  4.80           N  
ATOM    701  CA  LYS B 319     -18.097  25.837  -8.674  1.00  4.31           C  
ATOM    702  C   LYS B 319     -18.281  24.408  -8.180  1.00  3.72           C  
ATOM    703  O   LYS B 319     -19.378  23.976  -7.897  1.00  3.65           O  
ATOM    704  CB  LYS B 319     -18.706  26.802  -7.649  1.00  4.65           C  
ATOM    705  CG  LYS B 319     -18.095  26.504  -6.275  1.00  5.14           C  
ATOM    706  CD  LYS B 319     -16.566  26.502  -6.387  1.00  5.90           C  
ATOM    707  CE  LYS B 319     -16.094  27.811  -7.022  1.00  6.52           C  
ATOM    708  NZ  LYS B 319     -16.456  28.951  -6.135  1.00  7.34           N  
ATOM    709  H1  LYS B 319     -18.362  25.255 -10.645  1.00  5.07           H  


In [23]:
!tail -10 atoms_chain_A.txt

ATOM   1388  HD2 PRO B 359      -0.325  26.181 -12.984  1.00  4.68           H  
ATOM   1389  HD3 PRO B 359      -0.403  24.455 -12.580  1.00  4.50           H  
ATOM   1390  N   GLY B 360       3.874  27.142 -10.260  1.00  5.39           N  
ATOM   1391  CA  GLY B 360       4.396  28.465  -9.817  1.00  5.90           C  
ATOM   1392  C   GLY B 360       5.886  28.563 -10.146  1.00  6.72           C  
ATOM   1393  O   GLY B 360       6.291  27.987 -11.143  1.00  7.20           O  
ATOM   1394  OXT GLY B 360       6.598  29.211  -9.397  1.00  7.11           O  
ATOM   1395  H   GLY B 360       4.471  26.366 -10.297  1.00  5.67           H  
ATOM   1396  HA2 GLY B 360       3.860  29.253 -10.328  1.00  5.94           H  
ATOM   1397  HA3 GLY B 360       4.259  28.569  -8.751  1.00  5.99           H  


Nice!

## Finding files and with glob

In the above snippet of code, we extracted all atom records from a PDB file. We might want to do this (or some other operation) for many files. For example, the directory `~/git/data/` has four PDB files in it. For the present discussion, let's say we want to pull the sequence of chain A out of each PDB file.

The `glob` module from the standard library enables us to get a list of all files that match a pattern. In our case, we want all files matching `data/*.pdb`, where `*` is a **wild card character**, meaning that any matches of characters where `*` appears are allowed. Let's see what `glob.glob()` gives us.

In [60]:
file_list = glob.glob('/Users/jschaefer/git/bootcamp/data/*.pdb')

file_list

['/Users/jschaefer/git/bootcamp/data/1OLG.pdb',
 '/Users/jschaefer/git/bootcamp/data/1J6Z.pdb',
 '/Users/jschaefer/git/bootcamp/data/1FAG.pdb',
 '/Users/jschaefer/git/bootcamp/data/2ERK.pdb']

We have the four PDB files. We can now loop over them and pull out the sequences.

In [35]:
pwd

'/Users/jschaefer/git/bootcamp/lections'

In [61]:
import os
import glob
os.chdir('/Users/jschaefer/git/bootcamp/data/')

# Dictionary to hold sequences
seqs = {}

# Loop through all matching files
for file_name in file_list:
    # Extract PDB ID
    pdb_id = file_name[file_name.rfind('/')+1:file_name.rfind('.')]
    
    # Initialize sequence string, which we build as we go along
    seq = ''
    with open(file_name, 'r') as f:
        for line in f:
            if len(line) > 11 and line[:6] == 'SEQRES' and line[11] == 'A':
                seq += line[19:].rstrip() + ' '

    # Build sequence with dash-joined three letter codes
    seq = '-'.join(seq.split())

    # Store in the dictionary
    seqs[pdb_id] = seq



Let's take a look at what we got. We'll look at actin.

In [62]:
seqs

{'1OLG': 'LYS-LYS-LYS-PRO-LEU-ASP-GLY-GLU-TYR-PHE-THR-LEU-GLN-ILE-ARG-GLY-ARG-GLU-ARG-PHE-GLU-MET-PHE-ARG-GLU-LEU-ASN-GLU-ALA-LEU-GLU-LEU-LYS-ASP-ALA-GLN-ALA-GLY-LYS-GLU-PRO-GLY',
 '1J6Z': 'ASP-GLU-ASP-GLU-THR-THR-ALA-LEU-VAL-CYS-ASP-ASN-GLY-SER-GLY-LEU-VAL-LYS-ALA-GLY-PHE-ALA-GLY-ASP-ASP-ALA-PRO-ARG-ALA-VAL-PHE-PRO-SER-ILE-VAL-GLY-ARG-PRO-ARG-HIS-GLN-GLY-VAL-MET-VAL-GLY-MET-GLY-GLN-LYS-ASP-SER-TYR-VAL-GLY-ASP-GLU-ALA-GLN-SER-LYS-ARG-GLY-ILE-LEU-THR-LEU-LYS-TYR-PRO-ILE-GLU-HIC-GLY-ILE-ILE-THR-ASN-TRP-ASP-ASP-MET-GLU-LYS-ILE-TRP-HIS-HIS-THR-PHE-TYR-ASN-GLU-LEU-ARG-VAL-ALA-PRO-GLU-GLU-HIS-PRO-THR-LEU-LEU-THR-GLU-ALA-PRO-LEU-ASN-PRO-LYS-ALA-ASN-ARG-GLU-LYS-MET-THR-GLN-ILE-MET-PHE-GLU-THR-PHE-ASN-VAL-PRO-ALA-MET-TYR-VAL-ALA-ILE-GLN-ALA-VAL-LEU-SER-LEU-TYR-ALA-SER-GLY-ARG-THR-THR-GLY-ILE-VAL-LEU-ASP-SER-GLY-ASP-GLY-VAL-THR-HIS-ASN-VAL-PRO-ILE-TYR-GLU-GLY-TYR-ALA-LEU-PRO-HIS-ALA-ILE-MET-ARG-LEU-ASP-LEU-ALA-GLY-ARG-ASP-LEU-THR-ASP-TYR-LEU-MET-LYS-ILE-LEU-THR-GLU-ARG-GLY-TYR-SER-PHE-VAL-THR-TH

Excellent! Our function worked, and we now have the protein sequence.

## Computing environment

In [20]:
%load_ext watermark
%watermark -v -p jupyterlab

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.20.0

jupyterlab: 4.0.13

